In [1]:
!pip install fastapi uvicorn nest_asyncio pyngrok opencv-python onnxruntime tensorflow
!pip install python-multipart
!ngrok config add-authtoken 2w632UmskfhkiCHNqk8koimwg08_89pfMfZsEGcJj2q56zHkD

  Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl (375.9 MB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\Fayroz\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\client_channel\\lb_policy\\grpclb\\client_load_reporting_filter.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Fayroz\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Fayroz\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
'ngrok' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
!pip install pycaret


In [1]:
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
import shutil
import pandas as pd
import traceback
from pycaret.classification import load_model
from pose_feature_extractor import extract_features_from_video

app = FastAPI()

# Load PyCaret pipeline
model = load_model("my_first_pipeline4")

@app.post("/analyze_video/")
async def analyze_video(file: UploadFile = File(...)):
    try:
        video_path = "temp_video.mp4"
        with open(video_path, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)

        df = extract_features_from_video(video_path)
        df.to_csv("player_features.csv", index=False)

        prediction = model.predict(df)[0]

        return {
            "prediction": prediction,
            "features": df.iloc[0].to_dict()
        }

    except Exception as e:
        return JSONResponse(
            status_code=500,
            content={
                "error": str(e),
                "traceback": traceback.format_exc()
            }
        )

Transformation Pipeline and Model Successfully Loaded


In [ ]:
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
import shutil
import pandas as pd
import traceback
from pycaret.classification import load_model
from pose_feature_extractor import extract_features_from_video

app = FastAPI()

model = load_model("my_first_pipeline4")

@app.post("/process_chunk/")
async def process_chunk(file: UploadFile = File(...)):
    try:
        video_path = f"temp_chunks/{file.filename}"
        with open(video_path, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)

        df = extract_features_from_video(video_path)
        df.to_csv(f"features_csv/{file.filename}_features.csv", index=False)

        prediction = model.predict(df)[0]

        return {
            "filename": file.filename,
            "prediction": prediction,
            "features": df.iloc[0].to_dict()  
        }

    except Exception as e:
        return JSONResponse(
            status_code=500,
            content={
                "error": str(e),
                "traceback": traceback.format_exc()
            }
        )


In [2]:
import nest_asyncio
import uvicorn
from pyngrok import ngrok

nest_asyncio.apply()

# Open ngrok tunnel to port 8000
public_url = ngrok.connect(8000)
print("🚀 Public URL:", public_url)

# Run FastAPI app
uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [20489]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🚀 Public URL: NgrokTunnel: "https://ce65-35-245-212-108.ngrok-free.app" -> "http://localhost:8000"
INFO:     154.177.181.194:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     154.177.181.194:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     154.177.181.194:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     154.177.181.194:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     154.177.181.194:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     154.177.181.194:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     154.177.181.194:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     154.177.181.194:0 - "POST /analyze_video/ HTTP/1.1" 200 OK
INFO:     154.177.181.194:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     154.177.181.194:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     154.177.181.194:0 - "GET /docs HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [20489]
